In [61]:
import pandas as pd
import numpy as np
import re
import emoji
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer

nltk.download('words')
words = set(nltk.corpus.words.words())
stop_words = set(stopwords.words('english'))



[nltk_data] Downloading package words to
[nltk_data]     /Users/mosheborochov/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [62]:
df.head()

,id,text,created_at,hashtags,mentions,in_reply_to_user_id,user_id,retweeted,replied_to,quoted,is_anomaly
0,1354218419441315840,With today’s winter weather—and more in the fo...,2021-01-27 00:03:26,[CentralParkConservancy],[],NaN,96795310,NaN,NaN,NaN,0
1,1354218565977706496,RT @CentralParkNYC: With today’s winter weathe...,2021-01-27 00:04:00,[],[CentralParkNYC],NaN,1288324520487669760,1.354218e+18,NaN,NaN,0
2,1354219142048440320,@CentralParkNYC https://t.co/0DTEzyA1xx,2021-01-27 00:06:18,[],[CentralParkNYC],96795310.0,119252795,NaN,1.354218e+18,NaN,0
3,1354219554138968064,RT @CentralParkNYC: With today’s winter weathe...,2021-01-27 00:07:56,[],[CentralParkNYC],NaN,151539920,1.354218e+18,NaN,NaN,0
4,1354226585956716544,RT @CentralParkNYC: With today’s winter weathe...,2021-01-27 00:35:53,[],[CentralParkNYC],NaN,1342261723793907712,1.354218e+18,NaN,NaN,0


In [124]:
df = pd.read_json('../data/mstream_datasets/CentralParkNYC-2021-01-27-2021-02-06.json')
print(df['text'])
def stemming(words):
    ps=PorterStemmer()
    return [ps.stem(word) for word in words]

def remove_stop_words(words):
    return [c.lower() for c in words if c not in stop_words]

def lemmatizing(words):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]


def cleaner(tweet):
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet).split())
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    return tweet

def final_text(words):
     return ' '.join(words)

#df['text'] = df['text'].map(lambda x: cleaner(x))
df['text'] = df['text'].apply(lambda x:word_tokenize(x))
#df['text'] = df['text'].apply(lambda x:remove_stop_words(x))
#df['text'] = df['text'].apply(lambda x: lemmatizing(x))
#df['text'] = df['text'].apply(lambda x: stemming(x))
#df['text'] = df['text'].apply(lambda x:final_text(x))

0       With today’s winter weather—and more in the fo...
1       RT @CentralParkNYC: With today’s winter weathe...
2                 @CentralParkNYC https://t.co/0DTEzyA1xx
3       RT @CentralParkNYC: With today’s winter weathe...
4       RT @CentralParkNYC: With today’s winter weathe...
                              ...                        
3612    @yaelbt @BirdCentralPark @CentralParkNYC "Than...
3613    RT @sandro_pozzi: Y dónde van los patos y gans...
3614    @centralparknyc @nyc_reservoir @ Central Park ...
3615    RT @Antisomniac: The highlight of @CentralPark...
3616    RT @Antisomniac: Last bird post for the day fr...
Name: text, Length: 3617, dtype: object


In [125]:
df

,id,text,created_at,hashtags,mentions,in_reply_to_user_id,user_id,retweeted,replied_to,quoted,is_anomaly
0,1354218419441315840,"[With, today, ’, s, winter, weather—and, more,...",2021-01-27 00:03:26,[CentralParkConservancy],[],NaN,96795310,NaN,NaN,NaN,0
1,1354218565977706496,"[RT, @, CentralParkNYC, :, With, today, ’, s, ...",2021-01-27 00:04:00,[],[CentralParkNYC],NaN,1288324520487669760,1.354218e+18,NaN,NaN,0
2,1354219142048440320,"[@, CentralParkNYC, https, :, //t.co/0DTEzyA1xx]",2021-01-27 00:06:18,[],[CentralParkNYC],96795310.0,119252795,NaN,1.354218e+18,NaN,0
3,1354219554138968064,"[RT, @, CentralParkNYC, :, With, today, ’, s, ...",2021-01-27 00:07:56,[],[CentralParkNYC],NaN,151539920,1.354218e+18,NaN,NaN,0
4,1354226585956716544,"[RT, @, CentralParkNYC, :, With, today, ’, s, ...",2021-01-27 00:35:53,[],[CentralParkNYC],NaN,1342261723793907712,1.354218e+18,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
3612,1358200892051361792,"[@, yaelbt, @, BirdCentralPark, @, CentralPark...",2021-02-06 23:48:21,[],"[yaelbt, BirdCentralPark, CentralParkNYC]",20558567.0,412429578,NaN,1.358199e+18,NaN,0
3613,1358201019327516672,"[RT, @, sandro_pozzi, :, Y, dónde, van, los, p...",2021-02-06 23:48:51,[],"[sandro_pozzi, CentralParkNYC]",NaN,3078696420,1.358172e+18,NaN,NaN,0
3614,1358201096871763968,"[@, centralparknyc, @, nyc_reservoir, @, Centr...",2021-02-06 23:49:10,[],[CentralParkNYC],96795310.0,32035601,NaN,NaN,NaN,0
3615,1358203203863552000,"[RT, @, Antisomniac, :, The, highlight, of, @,...",2021-02-06 23:57:32,[],"[Antisomniac, CentralParkNYC]",NaN,1348657985782996992,1.358108e+18,NaN,NaN,0


In [126]:
df = df.explode('hashtags').explode('mentions').explode('text')
continuous_index = []
symbolic_index = ['hashtags', 'user_id', 'mentions', 'text']

df_continuous = df.loc[:, continuous_index]
df_symbolic = df.loc[:, symbolic_index]
df_label = df.loc[:, ['is_anomaly']]

In [127]:
df_symbolic.loc[:,'hashtags'].unique()
df_symbolic.loc[:,'user_id'].unique()
df_symbolic.loc[:,'mentions'].unique()
df_symbolic.loc[:,'text'].unique()


hashtags_dict = {}
for i, entry in enumerate(df_symbolic.loc[:,'hashtags'].unique()):
    hashtags_dict[entry] = i
    
user_id_dict = {}
for i, entry in enumerate(df_symbolic.loc[:,'user_id'].unique()):
    user_id_dict[entry] = i

mentions_dict = {}
for i, entry in enumerate(df_symbolic.loc[:,'mentions'].unique()):
    mentions_dict[entry] = i
    
text_dict = {}
for i, entry in enumerate(df_symbolic.loc[:,'text'].unique()):
    text_dict[entry] = i

In [128]:
df_symbolic.loc[:,'hashtags'] =  df_symbolic.loc[:,'hashtags'].map(hashtags_dict)
df_symbolic.loc[:,'user_id'] =  df_symbolic.loc[:,'user_id'].map(user_id_dict)
df_symbolic.loc[:,'mentions'] =  df_symbolic.loc[:,'mentions'].map(mentions_dict)
df_symbolic.loc[:,'text'] =  df_symbolic.loc[:,'text'].map(text_dict)

In [129]:
df_continuous.to_csv("CentralParkNYCnumeric.txt", index=False, header=False)
df_symbolic.to_csv("CentralParkNYCcateg.txt", index=False, header=False)
df_label.to_csv("CentralParkNYC_label.txt", index=False, header=False)
df.loc[:,'created_at'].to_csv("CentralParkNYCtime.txt", index=False, header=False)

In [130]:
df_symbolic.shape[0]

383255

In [131]:
text_file = open("CentralParkNYCnumeric.txt", "w")
df_symbolic.shape[0]
n = text_file.write('\n'*df_symbolic.shape[0])
text_file.close()

In [132]:
#df_label.groupby(['is_anomaly']).agg(['min', 'max', 'count', 'nunique'])
#df_label.groupby(['is_anomaly']).agg(['min', 'max', 'count', 'nunique'])
df['is_anomaly'].value_counts()

0    220105
1    163150
Name: is_anomaly, dtype: int64

In [133]:
57046/(74406+57046)

0.4339682926087089